In [1]:
import numpy as np

In [68]:
A = (np.arange(10)*2).reshape(5,2)

In [69]:
A = np.concatenate((A,np.array([[2,2]])))
A

array([[ 0,  2],
       [ 4,  6],
       [ 8, 10],
       [12, 14],
       [16, 18],
       [ 2,  2]])

In [70]:
N = A.shape[0]
A_temp = np.expand_dims(A,axis=-1)
ones_temp = np.ones((N,1,N))
print((A_temp@ones_temp).shape)
prod_temp = A_temp@ones_temp
prod_temp[0]

(6, 2, 6)


array([[0., 0., 0., 0., 0., 0.],
       [2., 2., 2., 2., 2., 2.]])

In [71]:
transposed = np.transpose(prod_temp,(0,2,1))
transposed[:,:,0]

array([[ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 4.,  4.,  4.,  4.,  4.,  4.],
       [ 8.,  8.,  8.,  8.,  8.,  8.],
       [12., 12., 12., 12., 12., 12.],
       [16., 16., 16., 16., 16., 16.],
       [ 2.,  2.,  2.,  2.,  2.,  2.]])

In [72]:
transposed_2 = np.transpose(transposed,(1,0,2))
transposed_2[:,:,0]

array([[ 0.,  4.,  8., 12., 16.,  2.],
       [ 0.,  4.,  8., 12., 16.,  2.],
       [ 0.,  4.,  8., 12., 16.,  2.],
       [ 0.,  4.,  8., 12., 16.,  2.],
       [ 0.,  4.,  8., 12., 16.,  2.],
       [ 0.,  4.,  8., 12., 16.,  2.]])

In [87]:
def measure_inter_dist(A):
    assert len(A.shape) == 2, "wrong shape"
    N = A.shape[0]
    d = A.shape[1]

    A_temp = np.expand_dims(A,axis=-1)
    ones_temp = np.ones((N,1,N))
    prod_temp = A_temp@ones_temp
    transposed = np.transpose(prod_temp,(0,2,1))
    transposed_2 = np.transpose(transposed,(1,0,2))
    sqr_dist = ((np.abs(transposed-transposed_2)**2)@np.ones((N,d,1)))[:,:,0]
    dist = np.sqrt(sqr_dist)
    return dist

# measure_inter_dist(A)

In [113]:
tb3_poses = np.array([[0,1,0],[0,100,0]]).T
cf_poses = np.array([[0,0,2],[2,2,2]]).T
radius_inter_robot_security_xyz = 1
flee_speed = 0.5

def apply_anticollision(robotNo, tb3_poses, cf_poses, vx, vy, vz, takeoff, land):
    all_poses = np.concatenate((cf_poses, tb3_poses), axis=-1)
    inter_dist_xyz = measure_inter_dist(all_poses.T)
    robots_at_risk = np.where(inter_dist_xyz[robotNo-1] <= radius_inter_robot_security_xyz)[0]
    
    if len(robots_at_risk) > 1:
        barycenter =  np.mean(all_poses[:,robots_at_risk],axis=-1)
        flee_vector = cf_poses[:,robotNo-1]-barycenter
        flee_vector = flee_vector/np.linalg.norm(flee_vector)
        flee_vector = flee_speed*flee_vector
        vx, vy, vz = flee_vector
        takeoff, land = False, False
    return vx, vy, vz, takeoff, land

print(cf_poses)
print(tb3_poses)
vx, vy, vz, takeoff, land = 0, 0, 0, False, False
print(apply_anticollision(1, tb3_poses, cf_poses, vx, vy, vz, takeoff, land))

[[0 2]
 [0 2]
 [2 2]]
[[  0   0]
 [  1 100]
 [  0   0]]
all_poses [[  0   2   0   0]
 [  0   2   1 100]
 [  2   2   0   0]]
[[  0.           2.82842712   2.23606798 100.019998  ]
 [  2.82842712   0.           3.          98.04080783]
 [  2.23606798   3.           0.          99.        ]
 [100.019998    98.04080783  99.           0.        ]]
(1,)
(0, 0, 0, False, False)


0